In [5]:
pip install tqdm


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 78 kB 4.5 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


### Have only tried with 50000 simulations - not sure if the logic is correct for the unique combinations here

In [24]:
import numpy as np
import itertools
import json
import os
from tqdm import tqdm

def load_decks(path):
    """Load deck data from .npy file and convert to binary strings."""
    data = np.load(path, allow_pickle = True)
    decks = data[:, 1]
    decks.tolist()
   
    decks = decks[:50000] #for testing making it shorter
    
    return decks

def score_deck(deck: str, seq1: str, seq2: str, deck_length: int) -> tuple[int, int, int, int]:
    """Score a single deck for both players."""
    p1_cards = p2_cards = 0
    p1_tricks = p2_tricks = 0
    pile = 2
    i = 0
    
    while i < deck_length:
        pile += 1
        current_sequence = deck[i:i+3]
        if current_sequence == seq1:
            p1_cards += pile
            p1_tricks += 1
            pile = 2
            i += 3
        elif current_sequence == seq2:
            p2_cards += pile
            p2_tricks += 1
            pile = 2
            i += 3
        else:
            i += 1
    
    return p1_cards, p2_cards, p1_tricks, p2_tricks

def calculate_winner(p1_cards: int,
                     p2_cards: int,
                     p1_tricks: int,
                     p2_tricks: int):

        '''Given the number of cards and tricks for each player, calculate who wins for cards and tricks, as well as draws for cards and tricks.
            If player one wins, the winner is set to 0. If player 2 wins, the winner is set to 1.
            Also indicates if there was a draw.

        Arguments:
            p1_cards (int): number of cards player 1 won
            p2_cards (int): number of cards player 2 won
            p1_tricks (int): number of tricks player 1 won
            p2_tricks (int): number of tricks player 2 won
        
        Output:
            cards_winner (int): specifies who won based on cards
            cards_draw (int): 1 if a draw occurred, 0 otherwise
            tricks_winner (int): specifies who won based on tricks
            tricks_draw (int): 1 if a draw occured, 0 otherwise'''
        cards_winner = 0
        cards_draw = 0
        tricks_winner = 0
        tricks_draw = 0

   
        if p1_cards > p2_cards:
            cards_winner = 1
        elif p1_cards == p2_cards:
            cards_draw = 1
        if p1_tricks > p2_tricks:
            tricks_winner = 1
        elif p1_tricks == p2_tricks:
            tricks_draw = 1
        return cards_winner, cards_draw, tricks_winner, tricks_draw


def generate_unique_combinations(sequences):
    """
    Generate unique combinations from the given sequences.
    Returns both orderings of each pair since order matters for player 1 vs player 2.
    """
    # Step 1: Generate all pairs of sequences without repetition
    unique_combinations = set()
    
    for bin1, bin2 in itertools.combinations(sequences, 2):        
        # Create the inverted version of the pair
        inverted_pair = ('{0:03b}'.format(~int(bin1, 2) & 7), '{0:03b}'.format(~int(bin2, 2) & 7))
        
        # Add both orderings of original pair
        unique_combinations.add((bin1, bin2))
        unique_combinations.add((bin2, bin1))
        
        # Add both orderings of inverted pair if it's not already there
        unique_combinations.add(inverted_pair)
        unique_combinations.add((inverted_pair[1], inverted_pair[0]))
    
    return unique_combinations

def complete_matrices(partial_results, sequences):
    """
    Complete the matrices for player 1's perspective using the computed values.
    When the players are flipped (i.e. looking at the opposite side of the matrix),
    player 1's wins become 0 (losses) while ties remain the same.
    
    Args:
        partial_results (dict): Dictionary containing partially filled matrices
        sequences (list): List of all possible sequences
    
    Returns:
        dict: Dictionary containing completed matrices
    """
    n_sequences = len(sequences)
    
    # Create complete matrices
    complete_cards_wins = np.zeros((n_sequences, n_sequences))
    complete_tricks_wins = np.zeros((n_sequences, n_sequences))
    complete_cards_ties = np.zeros((n_sequences, n_sequences))
    complete_tricks_ties = np.zeros((n_sequences, n_sequences))
    
    # Fill in the matrices
    for i in range(n_sequences):
        for j in range(n_sequences):
            if i != j:  # Skip diagonal 
                if j > i:  # Upper triangle??? 
                    complete_cards_wins[i][j] = partial_results['cards'][i][j]
                    complete_tricks_wins[i][j] = partial_results['tricks'][i][j]
                    complete_cards_ties[i][j] = partial_results['cards_ties'][i][j]
                    complete_tricks_ties[i][j] = partial_results['tricks_ties'][i][j]
                else:  # Lower triangle??
                    complete_cards_wins[i][j] = 1 - (partial_results['cards'][j][i] + partial_results['cards_ties'][j][i])
                    complete_tricks_wins[i][j] = 1 - (partial_results['tricks'][j][i] + partial_results['tricks_ties'][j][i])
                    # Ties 
                    complete_cards_ties[i][j] = partial_results['cards_ties'][j][i]
                    complete_tricks_ties[i][j] = partial_results['tricks_ties'][j][i]
    
    return {
        'cards': complete_cards_wins.tolist(),
        'tricks': complete_tricks_wins.tolist(),
        'cards_ties': complete_cards_ties.tolist(),
        'tricks_ties': complete_tricks_ties.tolist(),
        'n': partial_results['n']
    }

def process_all_decks(decks, deck_length = 52):
    """Process all decks and compute statistics for all sequence combinations."""
    sequences = ['000', '001', '010', '011', '100', '101', '110', '111']
    n_sequences = len(sequences)
    
    unique_combinations = generate_unique_combinations(sequences) #gives us 16 combos to use
    print(unique_combinations)
    
    # Initialize result arrays
    cards_wins = np.zeros((n_sequences, n_sequences))
    tricks_wins = np.zeros((n_sequences, n_sequences))
    cards_ties = np.zeros((n_sequences, n_sequences))
    tricks_ties = np.zeros((n_sequences, n_sequences))
    
    # Create sequence index mapping
    seq_to_idx = {seq: idx for idx, seq in enumerate(sequences)}
    
    total_decks = len(decks)
    deck_length_minus2 = deck_length - 2

    for deck in tqdm(decks, desc="Processing decks"):
        for seq1, seq2 in unique_combinations:
            if seq1 != seq2: 
                idx1 = seq_to_idx[seq1]  # Getting index of sequence 1
                idx2 = seq_to_idx[seq2]  # Getting index of sequence 2
                
                p1_cards, p2_cards, p1_tricks, p2_tricks = score_deck(deck, seq1, seq2, deck_length_minus2)
                cards_winner, cards_draw, tricks_winner, tricks_draw = calculate_winner(p1_cards, p2_cards, p1_tricks, p2_tricks)
    
            
                if cards_draw:
                     cards_ties[idx1][idx2] += 1
                elif cards_winner == 1:  # Player 1 wins
                    cards_wins[idx1][idx2] += 1
            
                if tricks_draw:
                    tricks_ties[idx1][idx2] += 1
                elif tricks_winner == 1:  # Player 1 wins
                    tricks_wins[idx1][idx2] += 1    
            

    # Convert to probabilities
    partial_results = {
        'cards': cards_wins / total_decks,
        'tricks': tricks_wins / total_decks,
        'cards_ties': cards_ties / total_decks,
        'tricks_ties': tricks_ties / total_decks,
        'n': total_decks
    }
    
    # Complete the matrices
    final_results = complete_matrices(partial_results, sequences)
    
    return final_results

def process_and_save_results2(input_path, output_folder='results'):
    """Process decks from input file and save results to output folder."""
    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)
    
    # Load and process decks
    print("Loading decks...")
    decks = load_decks(input_path)
    
    # Process all decks and get results
    print("Processing games...")
    results = process_all_decks(decks)
    
    # Save results
    output_path = os.path.join(output_folder, 'results.json')
    with open(output_path, 'w') as f:
        json.dump(results, f)
    
    return results

In [25]:
process_and_save_results2('/Users/student/Desktop/fall 2024/Automation/Project Penney/penney_final_revisions/data/deck_data.npy')

Loading decks...
Processing games...
{('000', '010'), ('100', '101'), ('001', '101'), ('111', '000'), ('101', '111'), ('110', '111'), ('011', '000'), ('111', '011'), ('111', '110'), ('110', '101'), ('011', '110'), ('100', '000'), ('000', '100'), ('001', '000'), ('010', '100'), ('111', '001'), ('000', '111'), ('100', '011'), ('100', '110'), ('010', '111'), ('110', '000'), ('001', '011'), ('101', '000'), ('000', '101'), ('011', '001'), ('001', '110'), ('010', '101'), ('101', '011'), ('111', '010'), ('101', '110'), ('100', '001'), ('110', '011'), ('011', '010'), ('010', '000'), ('111', '100'), ('110', '001'), ('001', '010'), ('101', '001'), ('100', '010'), ('011', '100'), ('000', '011'), ('000', '110'), ('010', '011'), ('010', '110'), ('011', '111'), ('101', '010'), ('110', '010'), ('001', '100'), ('000', '001'), ('111', '101'), ('010', '001'), ('011', '101'), ('101', '100'), ('100', '111'), ('001', '111'), ('110', '100')}


Processing decks: 100%|█████████████████| 50000/50000 [00:46<00:00, 1083.21it/s]


{'cards': [[0.0, 0.47286, 0.27584, 0.1766, 2e-05, 0.24994, 0.00938, 0.4816],
  [0.50742, 0.0, 0.8643, 0.95918, 0.00204, 0.92118, 0.4775, 0.98792],
  [0.70764,
   0.12298000000000009,
   0.0,
   0.4888,
   0.41812,
   0.49096,
   0.06966,
   0.72608],
  [0.80316, 0.03305999999999998, 0.48326, 0.0, 0.48554, 0.5559, 0.99742, 1.0],
  [0.99998,
   0.99756,
   0.5607800000000001,
   0.48492,
   0.0,
   0.48718,
   0.03632,
   0.80092],
  [0.72882,
   0.06479999999999997,
   0.48622,
   0.42188000000000003,
   0.48372000000000004,
   0.0,
   0.12366,
   0.70778],
  [0.98808,
   0.48006000000000004,
   0.9159,
   0.0021400000000000308,
   0.95572,
   0.8631,
   0.0,
   0.50666],
  [0.48968,
   0.009000000000000008,
   0.25260000000000005,
   0.0,
   0.17715999999999998,
   0.27646000000000004,
   0.47392,
   0.0]],
 'tricks': [[0.0,
   0.40714,
   0.21094,
   0.11294,
   0.00104,
   0.16532,
   0.00792,
   0.32418],
  [0.44043999999999994,
   0.0,
   0.80386,
   0.8853,
   0.02686,
   0.7954,


In [ ]:
#######

### Processing all 56, but in batches - reduced runtime to 11 minutes

In [22]:
import numpy as np
import itertools
import json
import os
from tqdm import tqdm

# Global constants
SEQUENCES = ['000', '001', '010', '011', '100', '101', '110', '111']
SEQ_TO_IDX = {seq: idx for idx, seq in enumerate(SEQUENCES)}
VALID_PAIRS = [(seq1, seq2) for seq1 in SEQUENCES for seq2 in SEQUENCES if seq1 != seq2]

def load_decks(path):
    """Load deck data from .npy file."""
    data = np.load(path, allow_pickle=True)
    return data[:, 1]

def score_deck(deck, seq1, seq2, deck_length):
    """Optimized scoring function."""
    p1_cards = p2_cards = 0
    p1_tricks = p2_tricks = 0
    pile = 2
    i = 0
    seq_len = 3
    
    while i < deck_length:
        pile += 1
        current = deck[i:i+seq_len]
        if current == seq1:
            p1_cards += pile
            p1_tricks += 1
            pile = 2
            i += seq_len
        elif current == seq2:
            p2_cards += pile
            p2_tricks += 1
            pile = 2
            i += seq_len
        else:
            i += 1
    
    return p1_cards, p2_cards, p1_tricks, p2_tricks

def calculate_winner(p1_cards, p2_cards, p1_tricks, p2_tricks):
    """Calculate winner status."""
    cards_winner = 0
    cards_draw = 0
    tricks_winner = 0
    tricks_draw = 0

    if p1_cards > p2_cards:
        cards_winner = 1
    elif p1_cards == p2_cards:
        cards_draw = 1
    if p1_tricks > p2_tricks:
        tricks_winner = 1
    elif p1_tricks == p2_tricks:
        tricks_draw = 1
    return cards_winner, cards_draw, tricks_winner, tricks_draw

def process_deck_batch(deck, valid_pairs, deck_length_minus2):
    """Process a single deck for all valid pairs."""
    n_sequences = len(SEQUENCES)
    batch_cards_wins = np.zeros((n_sequences, n_sequences))
    batch_tricks_wins = np.zeros((n_sequences, n_sequences))
    batch_cards_ties = np.zeros((n_sequences, n_sequences))
    batch_tricks_ties = np.zeros((n_sequences, n_sequences))
    
    for seq1, seq2 in valid_pairs:
        idx1 = SEQ_TO_IDX[seq1]
        idx2 = SEQ_TO_IDX[seq2]
        
        p1_cards, p2_cards, p1_tricks, p2_tricks = score_deck(deck, seq1, seq2, deck_length_minus2)
        cards_winner, cards_draw, tricks_winner, tricks_draw = calculate_winner(p1_cards, p2_cards, p1_tricks, p2_tricks)
        
        if cards_draw:
            batch_cards_ties[idx1][idx2] += 1
        elif cards_winner == 1:
            batch_cards_wins[idx1][idx2] += 1
        
        if tricks_draw:
            batch_tricks_ties[idx1][idx2] += 1
        elif tricks_winner == 1:
            batch_tricks_wins[idx1][idx2] += 1
            
    return batch_cards_wins, batch_tricks_wins, batch_cards_ties, batch_tricks_ties

def process_all_decks(decks, deck_length=52):
    """Process all decks with visible progress bar."""
    n_sequences = len(SEQUENCES)
    total_decks = len(decks)
    deck_length_minus2 = deck_length - 2
    
    # Initialize result arrays
    cards_wins = np.zeros((n_sequences, n_sequences))
    tricks_wins = np.zeros((n_sequences, n_sequences))
    cards_ties = np.zeros((n_sequences, n_sequences))
    tricks_ties = np.zeros((n_sequences, n_sequences))
    
    # Process each deck
    for deck in tqdm(decks, desc="Processing decks"):
        batch_cw, batch_tw, batch_ct, batch_tt = process_deck_batch(
            deck, VALID_PAIRS, deck_length_minus2)
        cards_wins += batch_cw
        tricks_wins += batch_tw
        cards_ties += batch_ct
        tricks_ties += batch_tt

    # Convert to probabilities and lists
    results = {
        'cards': (cards_wins / total_decks).tolist(),
        'tricks': (tricks_wins / total_decks).tolist(),
        'cards_ties': (cards_ties / total_decks).tolist(),
        'tricks_ties': (tricks_ties / total_decks).tolist(),
        'n': total_decks
    }
    
    return results

def process_and_save_results(input_path, output_folder='results'):
    """Process decks from input file and save results."""
    os.makedirs(output_folder, exist_ok=True)
    
    print("Loading decks...")
    decks = load_decks(input_path)
    
    print("Processing games...")
    results = process_all_decks(decks)
    
    output_path = os.path.join(output_folder, 'results.json')
    with open(output_path, 'w') as f:
        json.dump(results, f)
    
    return results

In [23]:
process_and_save_results('/Users/student/Desktop/fall 2024/Automation/Project Penney/penney_final_revisions/data/deck_data.npy')


Loading decks...
Processing games...


Processing decks: 100%|█████████████| 1000000/1000000 [11:18<00:00, 1473.44it/s]


{'cards': [[0.0,
   0.471181,
   0.277245,
   0.176197,
   9e-06,
   0.252084,
   0.009624,
   0.485966],
  [0.508744, 0.0, 0.863651, 0.955683, 0.001951, 0.918409, 0.478439, 0.988121],
  [0.706454, 0.123296, 0.0, 0.485003, 0.418689, 0.488421, 0.068325, 0.725582],
  [0.803865, 0.036044, 0.486251, 0.0, 0.484757, 0.558079, 0.997592, 0.999979],
  [0.999983, 0.997593, 0.559711, 0.486521, 0.0, 0.485953, 0.036419, 0.8033],
  [0.72634, 0.067817, 0.487961, 0.420119, 0.485389, 0.0, 0.123647, 0.705578],
  [0.987745, 0.476946, 0.91769, 0.001919, 0.955389, 0.863371, 0.0, 0.507734],
  [0.485146, 0.009331, 0.253137, 1.4e-05, 0.176676, 0.278278, 0.472292, 0.0]],
 'tricks': [[0.0,
   0.403947,
   0.211763,
   0.112831,
   0.001116,
   0.168263,
   0.008337,
   0.324818],
  [0.441419, 0.0, 0.802194, 0.88356, 0.026337, 0.793117, 0.410226, 0.967088],
  [0.663588, 0.115577, 0.0, 0.43066, 0.421744, 0.406848, 0.113285, 0.693034],
  [0.76947, 0.05138, 0.432875, 0.0, 0.431389, 0.442881, 0.935577, 0.994948],
  